In [ ]:
import sys as _sys
import os

currentPath = os.path.abspath(os.getcwd())

split = currentPath.split("Cshells")
if len(split)<2:
    print("Please rename the repository 'Cshells'")
    raise ValueError
pathToPythonScripts = os.path.join(split[0], "Cshells/python/")
pathToModels = os.path.join(split[0], "Cshells/data/models")
pathToOutputs = os.path.join(split[0], "Cshells/output")

_sys.path.insert(0, pathToPythonScripts)

In [ ]:
import MeshFEM
import ElasticRods

import average_angle_linkages
from bending_validation import suppress_stdout as so
import cshell_optimization
import elastic_rods
from linkage_vis import LinkageViewer, LinkageViewerWithSurface
import math
import mesh
import numpy as np
import pickle
import torch

from CShell import CShell
from CurvesDoFOptimizer import CurvesDoFOptimizer
from InteropGH import Interop
from open_average_angle_linkage import open_average_angle_linkage
import py_newton_optimizer
from RestQuantitiesOptimizer import RestQuantitiesOptimizer
from vis.fields import ScalarField
from VisUtils import ConvergencePlotsVisualizer, PlotStackedConvergencePlots

torch.set_default_dtype(torch.float64)
    
def ToNumpy(tensor):
    return tensor.cpu().detach().clone().numpy()

PI = math.pi

# Initialization

Get the flat and deployed linkages as `AverageAngleSurfaceAttractedLinkage`.

In [ ]:
modelName = "hexagon_x"

pathToFolderJSON = os.path.join(pathToOutputs, "{}/optimization".format(modelName))
if not os.path.exists(pathToFolderJSON):
    os.makedirs(pathToFolderJSON)

with open(os.path.join(pathToModels, modelName, "flat_initial.p"), 'rb') as f:
    flatLinkage = pickle.load(f)
    
with open(os.path.join(pathToModels, modelName, "deployed_initial.p"), 'rb') as f:
    deployedLinkage = pickle.load(f)
    
with open(os.path.join(pathToModels, modelName, "attraction_mesh.p"), 'rb') as f:
    attractionMesh = pickle.load(f)
    
additionalFeatJoints = [10, 91, 51,  9, 31, 70] # Joints on the central Hexagon

flatView = LinkageViewer(flatLinkage, width=768, height=480)
deployedView   = LinkageViewerWithSurface(
    deployedLinkage, mesh.Mesh(*(attractionMesh["V"], attractionMesh["F"])), 
    wireframeSurf=False, transparent=True, width=768, height=480
)

# Optimization rounds

In the [X-shell](https://julianpanetta.com/publication/xshells/) framework, the optimization pipeline consists in two steps: rest lengths solve and design optimization.

In [ ]:
def ApplyWeights(linkageOptimizer, dictWeights):
    linkageOptimizer.beta                     = dictWeights["beta"]
    linkageOptimizer.gamma                    = dictWeights["gamma"]
    linkageOptimizer.smoothing_weight         = dictWeights["smoothingWeight"]
    linkageOptimizer.rl_regularization_weight = dictWeights["rlRegWeight"]
    linkageOptimizer.invalidateAdjointState()

In [ ]:
newtonOptimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
newtonOptimizerOptions.gradTol = 1.0e-7
newtonOptimizerOptions.verbose = 1
newtonOptimizerOptions.beta = 1.0e-8
newtonOptimizerOptions.niter = 50
newtonOptimizerOptions.verboseNonPosDef = False

## Rest length solve

Keeping the joint positions of the flat layout fixed, we optimize the rest lengths so that the stress is minimized.

In [ ]:
additionalFixedFlatVars = flatLinkage.jointPositionDoFIndices()
additionalFixedDeployedVars = []

loRL = cshell_optimization.AverageAngleCShellOptimizationSAL(
    flatLinkage, deployedLinkage, newtonOptimizerOptions, 0.0, 
    optimizeTargetAngle=True, fixDeployedVars=False,
    additionalFixedFlatVars=additionalFixedFlatVars,
    additionalFixedDeployedVars=additionalFixedDeployedVars
)

loRL.setHoldClosestPointsFixed(False)

In [ ]:
flatView.show()

In [ ]:
deployedView.show()

In [ ]:
nSteps = 20
trustRegionScale = 1.0
optTol = 1.0e-3

screenshot = None

dictWeightsRL = {
    "beta": 0.0,
    "gamma": 1.0,
    "smoothingWeight": 0.0,
    "rlRegWeight": 0.0,
}

ApplyWeights(loRL, dictWeightsRL)

height = loRL.getDeployedLinkage().homogenousMaterial().crossSectionHeight
width  = loRL.getDeployedLinkage().homogenousMaterial().area / height
minRL  = 1.6 * width

rqoRL = RestQuantitiesOptimizer(loRL, "knitro_hvp.opt", minAngle=None, applyFlatnessConstraint=False, minRL=minRL)
optDP_RL, cocbRL = rqoRL.OptimizeDP(
    numSteps=nSteps, trustRegionScale=trustRegionScale, optTol=optTol, 
    useCB=True, maxEqSteps=50, computeGradMags=False, screenshot=screenshot,
    flatView=flatView, deployedView=deployedView
)
nIterationsRL = len(cocbRL.iterateData)

In [ ]:
cpvRL = ConvergencePlotsVisualizer(cocbRL)
cpvRL.PlotObjective(os.path.join(pathToFolderJSON, "optimValsRL.png"), "Convergence plot (objectives)", plotAll=True, wrtTime=True)
cpvRL.PlotConstraints(os.path.join(pathToFolderJSON, "optimConstraintsRL.png"), "Convergence plot (constraints)", wrtTime=True)
cpvRL.PlotGradMags(os.path.join(pathToFolderJSON, "optimGradMagsRL.png"), "Convergence plot (gradient magnitudes)", wrtTime=True)

# Design optimization

We now optimize the flat and deployed state jointly.

In [ ]:
additionalFixedFlatVars = []
additionalFixedDeployedVars = []

loDO = cshell_optimization.AverageAngleCShellOptimizationSAL(
    flatLinkage, deployedLinkage, newtonOptimizerOptions, 0.0, 
    optimizeTargetAngle=True, fixDeployedVars=False,
    additionalFixedFlatVars=additionalFixedFlatVars,
    additionalFixedDeployedVars=additionalFixedDeployedVars,
)

loDO.setHoldClosestPointsFixed(False)
loDO.setTargetSurface(attractionMesh["V"], attractionMesh["F"])
loDO.scaleJointWeights(jointPosWeight=0.1)
loDO.setTargetJointsPosition(attractionMesh["targetJP"].reshape(-1,))


In [ ]:
flatView.show()

In [ ]:
deployedView.show()

### No flatness

As opposed to the original paper, we decide not to enforce the flatness constraint in the flat state in a first optimization round. In this way, the energy can be reduced in both the flat and deployed state in a gentle way. Flatness is enforced in a second step.

In [ ]:
nSteps = 200
trustRegionScale = 1.0
optTol = 1.0e-3

screenshot = None

dictWeights = {
    "beta": 4.0e5,
    "gamma": 0.9,
    "smoothingWeight": 0.0,
    "rlRegWeight": 0.0,
}

ApplyWeights(loDO, dictWeights)
loDO.scaleJointWeights(
    jointPosWeight=0.1, featureMultiplier=10.0, 
    additional_feature_pts=additionalFeatJoints,
)
loDO.setHoldClosestPointsFixed(False)
rqo1 = RestQuantitiesOptimizer(loDO, "knitro_hvp.opt", minAngle=np.pi/16, applyFlatnessConstraint=False, minRL=minRL)
optDP, cocb1 = rqo1.OptimizeDP(
    numSteps=nSteps, trustRegionScale=trustRegionScale, optTol=optTol, 
    useCB=True, maxEqSteps=50, computeGradMags=False, screenshot=screenshot,
    flatView=flatView, deployedView=deployedView, honorbounds=0
)
nIterations1 = len(cocb1.iterateData)

In [ ]:
assert False

### Apply flatness

Now that the linkage has lower energy in its flat state, we attempt to apply the flatness constraint.

In [ ]:
nSteps = 1000
trustRegionScale = 1.0
optTol = 1.0e-3

screenshot = None

dictWeights = {
    "beta": 4.0e5,
    "gamma": 0.9,
    "smoothingWeight": 0.0,
    "rlRegWeight": 0.0,
}

ApplyWeights(loDO, dictWeights)
loDO.scaleJointWeights(
    jointPosWeight=0.1, featureMultiplier=10.0, 
    additional_feature_pts=additionalFeatJoints,
)
loDO.setHoldClosestPointsFixed(False)
rqo2 = RestQuantitiesOptimizer(loDO, "knitro_hvp.opt", minAngle=np.pi/16, applyFlatnessConstraint=True, minRL=minRL)
optDP, cocb2 = rqo2.OptimizeDP(
    numSteps=nSteps, trustRegionScale=trustRegionScale, optTol=optTol, 
    useCB=True, maxEqSteps=50, computeGradMags=False, screenshot=screenshot,
    flatView=flatView, deployedView=deployedView, honorbounds=0
)
nIterations2 = len(cocb2.iterateData)

In [ ]:
nIterations = nIterations1 + nIterations2
print("There has been {} iterations".format(nIterations))

In [ ]:
assert False

In [ ]:
cpv = ConvergencePlotsVisualizer(cocb1)
# cpv = ConvergencePlotsVisualizer(cocb2)
cpv.PlotObjective(os.path.join(pathToFolderJSON, "optimVals.png"), "Convergence plot (objectives)", plotAll=True, wrtTime=True)
cpv.PlotConstraints(os.path.join(pathToFolderJSON, "optimConstraints.png"), "Convergence plot (constraints)", wrtTime=True)
cpv.PlotGradMags(os.path.join(pathToFolderJSON, "optimGradMags.png"), "Convergence plot (gradient magnitudes)", wrtTime=True)

In [ ]:
tsf = linkageOptimizer.target_surface_fitter
l0 = np.linalg.norm(np.max(tsf.V, axis=0) - np.min(tsf.V, axis=0))
print("Initial target deviation")
for key in cpv.dpsMetric["TargetDeviationReport"][0].keys():
    print("{}: {:.2f}%".format(key, cpv.dpsMetric["TargetDeviationReport"][0][key] * 100 / l0))
print("\nFinal target deviation")
for key in cpv.dpsMetric["TargetDeviationReport"][-1].keys():
    print("{}: {:.2f}%".format(key, cpv.dpsMetric["TargetDeviationReport"][-1][key] * 100 / l0))

In [ ]:
pathToSaveFolder = os.path.join(pathToOutputs, model_name, "/optimization")

with open(os.path.join(pathToSaveFolder, "flat_optimized.p"), 'wb') as f:
    pickle.dump(flatLinkage, f)

with open(os.path.join(pathToSaveFolder, "deployed_optimized.p"), 'wb') as f:
    pickle.dump(deployedLinkage, f)
